In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
%cd /content/drive/My Drive/Disaster_Tweets

/content/drive/My Drive/Disaster_Tweets


# 1. Thêm các thư viện cần sử dụng

In [41]:
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from functools import reduce

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Đọc dữ liệu tập train, test vào 2 dataframe: `train_df` và `test_df`

In [42]:
train_df = pd.read_csv('./dataset/train.csv')
test_df = pd.read_csv('./dataset/test.csv')

# 2. Cài đặt các hàm để tiền xử lý dữ liệu

## 2.1 Hàm xóa các đường link URL

In [43]:
def remove_URL(text):
    url_pattern = re.compile(r'\s*https?://\S+|www\.\S+\s*')
    return url_pattern.sub(r'', text)
check = "@bbcmtd ablaze http://t.co/lHYXEOHY6C ab"
check = remove_URL(check)
print(check)
print(len(check))

@bbcmtd ablaze ab
17


#2.2. Hàm chuyển sang chữ in thường

In [44]:
def to_lower(text):
  return text.lower()

#2.3. Dùng contractions (chuyển các từ viết tắt của đại từ nhân xưng sang dạng đầy đủ)

In [45]:
!pip install contractions


In [46]:
import contractions
def use_contractions(text):
  return contractions.fix(text)

check = "I ain't haha because it's like english. I'll've @minh"
use_contractions(check)

'I are not haha because it is like english. I will have @minh'

## 2.4. Xóa dấu câu

In [47]:
import string
string.punctuation, string.digits

('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~', '0123456789')

In [48]:
# PUNCT_TO_REMOVE = str(string.punctuation + string.digits)
PUNCT_TO_REMOVE = str(string.punctuation)
def remove_punc_and_num(text):
    """custom function to remove the punctuation"""
    for token in PUNCT_TO_REMOVE:
        text = text.replace(token, "")
    return text.strip()

check = "I'm am Minh, Minh is so so 2x abc @minh err:"
check = remove_punc_and_num(check)
print(check)
print(len(check))

Im am Minh Minh is so so 2x abc minh err
40


## 2.5 Xóa các từ stopwords

In [49]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
STOPWORDS = stopwords.words('english')
print(len(STOPWORDS))

179


In [51]:
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in STOPWORDS])
check = "I'm am Minh, Minh is so so 2x abc"
check = remove_stopwords(check)
print(check)
print(len(check))

I'm Minh, Minh 2x abc
21


## 2.6. Xóa các ký hiệu emoticons and emojis

In [52]:
!pip install emot --upgrade
import emot
emot_obj = emot.emot()

In [53]:
def handle_emoticons(text, remove_emoticon=True):
    dict_emoticons = dict(zip(emot_obj.emoticons(text)['value'], emot_obj.emoticons(text)['mean']))
    res_emoticons =  dict(sorted(dict_emoticons.items(), key = lambda kv:len(kv[1]), reverse=True))
    for emoticon, mean in res_emoticons.items():
        if remove_emoticon:
            text = text.replace(emoticon, "")
        else:
            text = text.replace(emoticon, mean)
    return text.strip()

def handle_emojis(text, remove_emoji=True):
    for emoji, mean in zip(emot_obj.emoji(text)['value'], emot_obj.emoji(text)['mean']):
        if remove_emoji:
            text = text.replace(emoji, "")
        else:
            text = text.replace(emoji, mean.replace(":", ""))
    return text

check = "I :)) Troi oi la troi :3 arsonist :D 🤔 🤣"
check = handle_emoticons(check)
print(check)
print(len(check))

check = handle_emojis(check)
print(check)
print(len(check))

I  Troi oi la troi  arsonist  🤔 🤣
33
I  Troi oi la troi  arsonist   
31


## 2.7 Sửa lỗi các từ viết sai

In [54]:
!pip install autocorrect
from autocorrect import Speller
spell = Speller(lang='en')

In [55]:
def correct_spelling(text):
    return spell(text)

print(correct_spelling("precessing"))
print(correct_spelling("helleo"))


processing
hello


## 2.8 Stemming and Lemmatization

In [56]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    for i in range(0,len(word_tokens)):
        word_tokens[i] = lemmatizer.lemmatize(word_tokens[i], 'v')
    return ' '.join([word for word in word_tokens])

print(lemmatize("running"))
print(lemmatize("I am a student"))

run
I be a student


## 2.9 Giải thích các từ viết tắt thông thường

In [57]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [58]:
def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word


def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

## 2.10 Xóa bỏ các ký tự không nằm trong bộ từ điển ASCII

In [59]:
def KeepAscii(text):
  return text.encode('ascii', 'ignore').decode()

## 2.11 Xóa bỏ các khoảng trắng trong câu xuất hiện không hợp lệ

In [60]:
def removeSuddenSpace(text):
    text = re.sub(r'\w*\d+\w*', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\s[^\w\s]\s', '', text)
    return text.strip()

removeSuddenSpace("  I       am; Minh      Hoang  ")

'I am; Minh Hoang'

# 3. Tiền xử lý

In [61]:
def preprocessing_csv(df):
    returned_df = df.copy()
    dict_func = [
        to_lower,
        KeepAscii,
        remove_URL,
        use_contractions,
        convert_abbrev_in_text,
        remove_punc_and_num,
        removeSuddenSpace,
        # remove_invalid_char,
        # remove_stopwords,
        # handle_emojis,
        # handle_emoticons,
        # correct_spelling,
        # remove_mention,
        # lemmatize
    ]
    for func in dict_func:
        returned_df['text'] = returned_df['text'].apply(lambda x: func(x))

    returned_df['keyword'] = returned_df['keyword'].str.replace('%20', ' ')

    return returned_df

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def group_similar_texts_with_same_loc(df, threshold):
    # Convert the text data into numerical features using TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df["text"])

    # Calculate the pairwise cosine similarity between documents
    cosine_sim = cosine_similarity(tfidf_matrix)

    # Group similar documents with the given threshold and keep the row with the highest frequency of target
    groups = {}
    for i in range(len(df)):
        group_id = None
        for j in range(i):
            if cosine_sim[i,j] > threshold and df['location'][i] == df['location'][j]:
                if group_id is None:
                    group_id = j
                elif df["target"][j] > df["target"][group_id]:
                    group_id = j
        if group_id is None:
            group_id = i
        groups.setdefault(group_id, []).append(i)

    # Create a new DataFrame with the grouped data and the most common target in each group
    grouped_data = []
    for group in groups.values():
        target_freq = df.loc[group]["target"].value_counts()
        most_common_target = target_freq.index[0]
        representative = df.loc[(df["target"] == most_common_target) & (df.index.isin(group))].iloc[0]
        grouped_data.append({
            "id": representative["id"],
            "keyword": representative["keyword"],
            "location": representative["location"],
            "text": ", ".join(df.loc[group]["text"]),
            "target": most_common_target
        })
    grouped_df = pd.DataFrame(grouped_data)

    return grouped_df

In [63]:
df = preprocessing_csv(train_df)

# 4. Loại bỏ các dữ liệu trùng nhau

In [64]:
# Similar tweet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def out_duplicate_text_with_same_loc(df, outFile):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['text'])

    # Calculate the cosine similarity between each pair of text data
    cosine_sim = cosine_similarity(tfidf_matrix)

    # Define a threshold for similarity score
    threshold = 0.7

    groups = {}
    for i in range(len(df)):
        group_id = None
        for j in range(i):
            if cosine_sim[i,j] > threshold and df['location'][i] == df['location'][j]:
                if group_id is None:
                    group_id = j
                elif df["target"][j] > df["target"][group_id]:
                    group_id = j
        if group_id is None:
            group_id = i
        groups.setdefault(group_id, []).append(i)

    count = 0
    same_id = []
    conflict_id = []

    with open(outFile, "w") as f:
        for group in groups.values():
            if(len(group) > 1):
                target_list = []
                list_id = []
                f.write(f'Group {count}:')
                for index in group:
                    target_list.append(df.iloc[index]["target"])
                    list_id.append(df.iloc[index]["id"])
                    f.write(f' - id={df.iloc[index]["id"]}, {df.iloc[index]["text"]}, target={df.iloc[index]["target"]}, location={df.iloc[index]["location"]}, key={df.iloc[index]["keyword"]}')
                    f.write('\n')
                if len(set(target_list)) > 1:
                  conflict_id.append(list_id)
                else:
                  same_id.append(list_id)

                count = count + 1
    return same_id, conflict_id


Dữ liệu gặp 2 vấn đề:
- Vấn đề 1 - `same_id`: Các dòng dữ liệu có nội dung giống nhau, nhãn target cũng giống nhau

- Vấn đề 2 - `conflict_id`: các dòng dữ liệu có nội dung giống nhau, nhãn target khác nhau.

Cách giải quyết:
- Vấn đề 1: Xóa tất cả, giữ lại một mẫu dữ liệu.
- Vấn đề 2: Điều tra, và vote nhãn cho các dòng dữ liệu bị mâu thuẫn này, sau đó xóa toàn bộ và giữ lại một mẫu dữ liệu.

In [65]:
same_id, conflict_id = out_duplicate_text_with_same_loc(df, "temp")

In [66]:
deleted_id = []
for id in same_id:
  deleted_id.extend(id[:-1])

In [67]:
print(conflict_id)

[[353, 390], [446, 447, 462], [560, 574], [796, 822], [805, 823, 3704, 3718, 3719, 3722, 3738, 3742, 3747], [1402, 1409], [1688, 1715], [1723, 1922], [1752, 1968], [1760, 1950], [1771, 1773], [2921, 2929], [3073, 3079], [3403, 3406, 3414], [3582, 3589], [3879, 3889, 3894], [4068, 4072, 4076, 4077], [4080, 4092], [4499, 4507], [5564, 5565, 5577, 5595, 5598, 5599, 5603], [5580, 5588, 5592], [5890, 5898, 5921], [6169, 6171], [6217, 6219], [6220, 6223], [6366, 6382], [6799, 6800, 6804, 6807, 6808, 6809, 6813], [6819, 6823, 6824, 6830, 6832, 6834, 6837, 6844, 6847], [6854, 6882], [7223, 7230, 7241], [7362, 7386], [7410, 7425], [7602, 7626], [7773, 7790], [8608, 8623], [8902, 8912, 8924], [9054, 9064], [9961, 10004]]


In [68]:
deleted_id.extend([390, 462, 560, 796, 805, 841, 1715, 1723, 1968, 1760, 1773, 2929, 3073, 3079, 3414, 3582, 3589, 3879,  4076, 4072, 4077, 4080, 4507, 5564, 5565, 5577, 5595, 5598, 5599, 5580, 5588,
                  5890, 5898, 6171, 6219, 6223, 6366, 6800, 6804, 6807, 6808, 6809, 6813, 6823, 6824, 6830, 6832, 6834, 6837, 6844, 6847, 6882, 7223, 7230, 7362, 7425, 7602, 7773, 8608, 8912, 8924, 9054, 9064, 9961
                  ])

In [69]:
print(len(deleted_id))

217


In [70]:
df[df.id == 7295]

,id,keyword,location,text,target
5115,7295,nuclear reactor,NaN,,0


In [71]:
df = df[~df.id.isin(deleted_id)]

In [72]:
df.loc[df.text == '', 'text'] = "UNK"


In [73]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,NaN,people receive wildfires evacuation orders in ...,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,ariaahrary thetawniest the out of control wild...,1
7610,10871,NaN,NaN,coordinated universal time s of volcano hawaii,1
7611,10872,NaN,NaN,police investigating after an ebike collided w...,1


In [74]:
df.to_csv('./preprocessing/light_pre_train_4.csv', index = False)

In [75]:
test_df = preprocessing_csv(test_df)
# df_test =preprocessing_csv_2(df_test)
test_df.loc[test_df.text == '', 'text'] = "UNK"

In [76]:

test_df.to_csv('./preprocessing/light_pre_test_4.csv', index = False)